In [1]:
from hero_db_utils.datamodels import datamodel
import dotenv
dotenv.load_dotenv(override=True);

In [2]:
from hero_db_utils.datamodels.fields import AutoSerialField

@datamodel
class Triangle:
    sidea:float
    sideb:float
    sidec:int
    id:AutoSerialField() = None

    @property
    def is_equilateral(self):
        return self.sidea == self.sideb == self.sidec
    
    @property
    def sides(self):
        return [self.sidea, self.sideb, self.sidec]


In [3]:
mytriangles = Triangle.collection([Triangle(5, 5, "5"), Triangle(10,5,2), Triangle(2,2,2)])
mytriangles.asdf()

,sidea,sideb,sidec,id
0,5.0,5.0,5.0,NaN
1,10.0,5.0,2.0,NaN
2,2.0,2.0,2.0,NaN


In [26]:
mytriangles.save()

In [5]:
Triangle.objects().all()

,id,sidea,sideb,sidec
0,1,5.0,5.0,5.0
1,2,10.0,5.0,2.0
2,3,2.0,2.0,2.0


In [6]:
Triangle(1,1,2).save()

Triangle(sidea=1.0, sideb=1.0, sidec=2, id=None)

In [7]:
Triangle.objects().all()

,id,sidea,sideb,sidec
0,1,5.0,5.0,5.0
1,2,10.0,5.0,2.0
2,3,2.0,2.0,2.0
3,4,1.0,1.0,2.0


In [8]:
model, created = Triangle.objects().get_or_create(sidea=1, sideb=1, sidec=2)
model.data

sidea    1.0
sideb    1.0
sidec    2.0
id       NaN
dtype: float64

In [9]:
model, created = Triangle.objects().get_or_create(sidea=12, sideb=13, sidec=22)
created

True

In [10]:
Triangle.objects().all()

,id,sidea,sideb,sidec
0,1,5.0,5.0,5.0
1,2,10.0,5.0,2.0
2,3,2.0,2.0,2.0
3,4,1.0,1.0,2.0
4,5,1.0,1.0,2.0
5,6,12.0,13.0,22.0
